In [1]:
import psycopg2
from psycopg2 import OperationalError, ProgrammingError
import pandas as pd

In [2]:
def connect(conn_params):
    conn = None
    try:
        print('Connecting to the PostgreSQL...........')
        conn = psycopg2.connect(conn_params)
        print("Connected successfully.")
        
    except OperationalError as err:
        print("Connection error ",err)        
        # set the connection to 'None' in case of error
        conn = None
    return conn

In [3]:
conn = connect("dbname=postgres user=postgres password=ashish")

Connecting to the PostgreSQL...........
Connected successfully.


In [4]:
def pandaSql(q):
    data = None
    try:
        data = pd.read_sql(q,conn)
    except Exception as err:
        print("Error occured :",err)
    finally:
        return data
    
def queryAll(cursor,q):
    data = None
    try:
        cursor.execute(q)
        data = cursor.fetchall()
    except ProgrammingError as err:
        print("Check your query String before running")
        print(err)
    except:
        print("Something went wrong..")
    return data

def executeQuery(cursor,q,d = None):   
    try:
        if d == None:
            cursor.execute(q)
        else:
            cursor.execute(q,d)
    except ProgrammingError as err:
        cursor.execute("ROLLBACK")
        print("Check your query String or data provided before running")
        print(err)
    except:
        cursor.execute("ROLLBACK")
        print("Something went wrong....")
    else:
        print("Query Successful")

In [5]:
cursor = conn.cursor()

In [6]:
query1 = """
Create Type statusenum as ENUM('completed', 
'cancelled_by_driver', 
'cancelled_by_client');
Create Type roleenum as ENUM('client', 'driver', 'partner');
"""
executeQuery(cursor,query1)

Check your query String or data provided before running
type "statusenum" already exists



In [7]:
query2 = """
Create table If Not Exists Trips (id integer, client_id integer, driver_id integer,
city_id integer, status statusenum,request_at varchar(50));
Create table If Not Exists Users (users_id integer, banned varchar(50),
role roleenum);
"""
executeQuery(cursor,query2)

Query Successful


In [8]:
query3 = """
TRUNCATE Table Trips;
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('1', '1', '10', '1', 'completed', '2013-10-01');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('2', '2', '11', '1', 'cancelled_by_driver', '2013-10-01');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('3', '3', '12', '6', 'completed', '2013-10-01');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('4', '4', '13', '6', 'cancelled_by_client', '2013-10-01');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('5', '1', '10', '1', 'completed', '2013-10-02');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('6', '2', '11', '6', 'completed', '2013-10-02');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('7', '3', '12', '6', 'completed', '2013-10-02');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('8', '2', '12', '12', 'completed', '2013-10-03');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('9', '3', '10', '12', 'completed', '2013-10-03');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('10', '4', '13', '12', 'cancelled_by_driver', '2013-10-03');
"""
executeQuery(cursor,query3)

Query Successful


In [9]:
query4 = "Select * from Trips"
queryAll(cursor,query4)

[(1, 1, 10, 1, 'completed', datetime.date(2013, 10, 1)),
 (2, 2, 11, 1, 'cancelled_by_driver', datetime.date(2013, 10, 1)),
 (3, 3, 12, 6, 'completed', datetime.date(2013, 10, 1)),
 (4, 4, 13, 6, 'cancelled_by_client', datetime.date(2013, 10, 1)),
 (5, 1, 10, 1, 'completed', datetime.date(2013, 10, 2)),
 (6, 2, 11, 6, 'completed', datetime.date(2013, 10, 2)),
 (7, 3, 12, 6, 'completed', datetime.date(2013, 10, 2)),
 (8, 2, 12, 12, 'completed', datetime.date(2013, 10, 3)),
 (9, 3, 10, 12, 'completed', datetime.date(2013, 10, 3)),
 (10, 4, 13, 12, 'cancelled_by_driver', datetime.date(2013, 10, 3))]

In [10]:
query5 = """
Truncate table Users;
insert into Users (users_id, banned, role) values ('1', 'No', 'client');
insert into Users (users_id, banned, role) values ('2', 'Yes', 'client');
insert into Users (users_id, banned, role) values ('3', 'No', 'client');
insert into Users (users_id, banned, role) values ('4', 'No', 'client');
insert into Users (users_id, banned, role) values ('10', 'No', 'driver');
insert into Users (users_id, banned, role) values ('11', 'No', 'driver');
insert into Users (users_id, banned, role) values ('12', 'No', 'driver');
insert into Users (users_id, banned, role) values ('13', 'No', 'driver');
"""
executeQuery(cursor,query5)

Something went wrong....


In [11]:
query6 = """
ALTER TABLE Trips
  ADD CONSTRAINT trip_pk 
    PRIMARY KEY (id);
ALTER TABLE USERS
    ADD CONSTRAINT user_pk
    PRIMARY KEY (users_id)
"""
executeQuery(cursor,query6)

Check your query String or data provided before running
multiple primary keys for table "trips" are not allowed



In [12]:
query7 = """
ALTER TABLE Trips
    ADD CONSTRAINT fk_trip_client FOREIGN KEY (client_id) REFERENCES Users (users_id);
ALTER TABLE Trips
    ADD CONSTRAINT fk_trip_driver FOREIGN KEY (driver_id) REFERENCES Users (users_id);
"""
executeQuery(cursor,query7)

Check your query String or data provided before running
constraint "fk_trip_client" for relation "trips" already exists



In [13]:
query8 = """
SELECT *
FROM 
   information_schema.columns
WHERE
    table_name='trips'
"""

In [14]:
pandaSql(query8)

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,postgres,public,trips,id,1,None,NO,integer,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,postgres,public,trips,client_id,2,None,YES,integer,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,postgres,public,trips,driver_id,3,None,YES,integer,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,postgres,public,trips,city_id,4,None,YES,integer,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,postgres,public,trips,status,5,None,YES,USER-DEFINED,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES
5,postgres,public,trips,request_at,6,None,YES,date,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES


In [15]:
query = "SELECT * from Trips"
pandaSql(query)

,id,client_id,driver_id,city_id,status,request_at
0,1,1,10,1,completed,2013-10-01
1,2,2,11,1,cancelled_by_driver,2013-10-01
2,3,3,12,6,completed,2013-10-01
3,4,4,13,6,cancelled_by_client,2013-10-01
4,5,1,10,1,completed,2013-10-02
5,6,2,11,6,completed,2013-10-02
6,7,3,12,6,completed,2013-10-02
7,8,2,12,12,completed,2013-10-03
8,9,3,10,12,completed,2013-10-03
9,10,4,13,12,cancelled_by_driver,2013-10-03


In [16]:
query = "SELECT * from users"
pandaSql(query)

,users_id,banned,role
0,1,No,client
1,2,Yes,client
2,3,No,client
3,4,No,client
4,10,No,driver
5,11,No,driver
6,12,No,driver
7,13,No,driver


In [17]:
varcharTodate = """
ALTER TABLE Trips ALTER COLUMN request_at TYPE DATE 
using to_date(request_at, 'YYYY-MM-DD');
"""
executeQuery(cursor,varcharTodate)

Check your query String or data provided before running
function to_date(date, unknown) does not exist
LINE 3: using to_date(request_at, 'YYYY-MM-DD');
              ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.



In [18]:
query = """
WITH needed_trips AS (
    SELECT t.*,
    u1.banned as banned_client,
    u2.banned as banned_driver
    FROM Trips t 
        INNER JOIN Users u1 ON (t.client_id = u1.users_id)
        INNER JOIN Users u2 ON (t.driver_id = u2.users_id)
    WHERE
        request_at >= '2013-10-01' AND
        request_at <= '2013-10-03'
    )
SELECT t.*,
    count(*) OVER (PARTITION BY t.request_at) as total_request_today
    FROM needed_trips t
    WHERE
        (t.banned_client = 'No' AND t.banned_driver = 'No')
"""
pandaSql(query)

,id,client_id,driver_id,city_id,status,request_at,banned_client,banned_driver,total_request_today
0,1,1,10,1,completed,2013-10-01,No,No,3
1,4,4,13,6,cancelled_by_client,2013-10-01,No,No,3
2,3,3,12,6,completed,2013-10-01,No,No,3
3,7,3,12,6,completed,2013-10-02,No,No,2
4,5,1,10,1,completed,2013-10-02,No,No,2
5,9,3,10,12,completed,2013-10-03,No,No,2
6,10,4,13,12,cancelled_by_driver,2013-10-03,No,No,2


In [19]:
query = """
WITH all_trips AS (
    SELECT t.request_at,
    count(*)
    FROM Trips t 
        INNER JOIN Users u1 ON (t.client_id = u1.users_id)
        INNER JOIN Users u2 ON (t.driver_id = u2.users_id)
    WHERE
        request_at >= '2013-10-01' AND
        request_at <= '2013-10-03' AND
        (u1.banned = 'No' AND u2.banned = 'No')
    GROUP BY t.request_at
    ), 
    cancelled_trips AS (
    SELECT t.request_at,
    count(*)
    FROM Trips t 
        INNER JOIN Users u1 ON (t.client_id = u1.users_id)
        INNER JOIN Users u2 ON (t.driver_id = u2.users_id)
    WHERE
        request_at >= '2013-10-01' AND
        request_at <= '2013-10-03' AND
        (u1.banned = 'No' AND u2.banned = 'No') AND
        t.status <> 'completed'
    GROUP BY t.request_at
    )
    Select 
        a.request_at,
        COALESCE(CAST(c.count AS DECIMAL), 0)/CAST(a.count AS DECIMAL)
            AS cancel_to_trips_ratio
        FROM all_trips a
            LEFT JOIN cancelled_trips c ON (a.request_at = c.request_at)
"""
pandaSql(query)

,request_at,cancel_to_trips_ratio
0,2013-10-01,0.333333
1,2013-10-02,0.000000
2,2013-10-03,0.500000
